# Load Functions

## 0. imports

In [ ]:
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.utils import ImageReader
import os
import mne
import json

from utils import *
from eeg_qc import compute_eeg_pipeline, test_eeg_pipeline
from ecg_qc import ecg_qc 
from eda_qc import eda_qc
from rsp_qc import *
from mic_qc import *
from lsl_problem import *
from et_qc import *
from webcam_qc import *
import matplotlib

In [ ]:
subject = "P5871751"
xdf_filename = f'/Users/bryan.gonzalez/CUNY_subs/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'

#subject = "P5182010"
#xdf_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'

#xdf_filename = #Apurva's xdf file

video_filename = '/'.join(xdf_filename.split('/')[:-1])+ f'/sub-{subject}_task-CUNY_run-001_video.avi'


In [ ]:
stim_df = import_stim_data(xdf_filename)

In [ ]:
stim_df

# Get Metrics

## 1. EEG

In [ ]:
#Compute the eeg vars
eeg_vars, raw_cleaned, ica, eeg_df = compute_eeg_pipeline(xdf_filename, 
                                                          stim_df=stim_df, 
                                                          task='RestingState')


### Manual Artifact Removal

In [ ]:
fig = ica.plot_components( title='ICA Components')
# Save the ICA plot
# change the figure size and save it
fig.set_size_inches(10, 8)
fig.tight_layout()
fig.savefig(f'report_images/{subject}_eeg_ica_components.png', bbox_inches='tight')

In [ ]:
ica.plot_properties(raw_cleaned, picks=range(ica.n_components_)) # This exact component number probably won't work if you recompute ICA


In [ ]:
ica.plot_overlay(raw_cleaned, picks=[0,3,10,11,13,14,18,19])

In [ ]:
ica.exclude = [0,3,10,11,13,14,18,19] # these are the components that we want to exclude
ica.apply(raw_cleaned)

In [ ]:
eeg_vars['components_excluded'] = ica.exclude

### Generate figures

In [ ]:
raw_cleaned.annotations.delete([i for i, desc in enumerate(raw_cleaned.annotations.description) if desc == 'blink' or desc == 'BAD_muscle'])
fig = raw_cleaned.plot(show_scrollbars=False,
                        show_scalebars=False,events=None, start=0, 
                        duration=200,n_channels=50, scalings=.35e-4, color='k', title='EEG Data after ICA')

fig.savefig(f'report_images/{subject}_eeg_cleaned.png', dpi=300, bbox_inches='tight')


fig = raw_cleaned.plot_psd(fmax=50, average=False, show=True)
fig.savefig(f'report_images/{subject}_eeg_cleaned_psd.png', dpi=300, bbox_inches='tight')

In [ ]:
raw_cleaned_pathname = '/'.join(xdf_filename.split('/')[:-1]) + f'/sub-{subject}_ses-S001_task-CUNY_run-001_eeg_clean.fif'
raw_cleaned.save(raw_cleaned_pathname, overwrite=True)


eeg_print = {
    'Bad channels before robust reference': eeg_vars['bad_channels_before'], 
    'Interpolated channels': eeg_vars['interpolated_channels'], 
    'Bad channels after interpolation': eeg_vars['bad_channels_after'], 
    'Percent Good before artifact removal': f"{eeg_vars['percent_good']:.4}%",
    'Artifactual Components Excluded': ica.exclude
}

## 2. ECG

In [ ]:
[ecg_vars, ecg_plt, ps_df] = ecg_qc(xdf_filename = xdf_filename, stim_df = stim_df, task='RestingState')


In [ ]:
ecg_vars.keys()

In [ ]:
ecg_print = {
    "Effective sampling rate": f'{ecg_vars["sampling_rate"]:.4f} Hz', 
    "Signal to Noise Ratio": f'{ecg_vars["SNR"]:.4f} dB',
    "Average heart rate": f'{ecg_vars["average_heart_rate"]:.4f} bpm',
    "Kurtosis signal quality index (kSQI)": f'{ecg_vars["kurtosis_SQI"]:.4f}',
    "Power spectrum distribution (pSQI)": f'{ecg_vars["power_spectrum_distribution_SQI"]:.4f} mV²/Hz',
    "Relative power in baseline (basSQI)": f'{ecg_vars["relative_baseline_power_sqi"]:.4f}%'
}

In [ ]:
ecg_print

## 3. EDA

In [ ]:
[eda_vars, eda_plt1, eda_plt2, ps_df] = eda_qc(xdf_filename = xdf_filename, stim_df = stim_df, task= 'RestingState')

In [ ]:
eda_vars.keys()

In [ ]:
eda_print = {
    "Effective sampling rate": f'{eda_vars["sampling_rate"]:.4f} Hz', 
    "Signal to noise ratio": f'{eda_vars["snr"]:.4f} dB',
    "Signal integrity check": f'{eda_vars["signal_integrity_check"]:.4f}%',
    "Average skin conductance level": f'{eda_vars["average_scl"]:.4f} mS',
    "Skin conductance level std": f'{eda_vars["scl_sd"]:.4f} mS',
    "Skin conductance level coefficient of variation": f'{eda_vars["scl_cv"]:.4f}%',
    "Average amplitude of skin conductance response": f'{eda_vars["average_scr_amplitude"]:.4f} mS',
    "Skin conductance response validity": f'{eda_vars["sc_validity"]:.4f} %' # this will need to be changed to [scr_validity]
}

## 4. RSP

In [ ]:
rsp_vars, ps_df = rsp_qc(xdf_filename=xdf_filename, stim_df=stim_df, task='RestingState')

In [ ]:
rsp_print = {
    "Effective sampling rate": f'{rsp_vars["sampling_rate"]:.4f} Hz', 
    "Signal to noise ratio": f'{rsp_vars["rsp_snr"]:.4f} dB',
    "Breath amplitude mean": f'{rsp_vars["breath_amplitude_mean"]:.4f} V',
    "Breath amplitude std": f'{rsp_vars["breath_amplitude_std"]:.4f} V',
    "Breath amplitude range": f'{rsp_vars["breath_amplitude_range"]} V',
    "Respiration rate mean": f'{rsp_vars["rsp_rate_mean"]:.4f} bpm',
    "Respiration rate std": f'{rsp_vars["rsp_rate_std"]:.4f} bpm', 
    "Respiration rate range": f'{rsp_vars["rsp_rate_range"]} bpm', 
    "Peak to peak interval mean": f'{rsp_vars["ptp_mean"]:.4f} sec', 
    "Peak to peak interval std": f'{rsp_vars["ptp_std"]:.4f} sec', 
    "Peak to peak interval range": f'{rsp_vars["ptp_range"]} sec', 
    "Baseline drift": f'{rsp_vars["baseline_drift"]:.4f} V', 
    "Autocorrelation at typical breath cycle": f'{rsp_vars["autocorrelation"]:.4f}'
}

## 5. Mic

In [ ]:
mic_vars, mic_df = mic_qc(xdf_filename=xdf_filename, stim_df=stim_df)

In [ ]:
mic_print = {
    "Effective sampling rate": f'{mic_vars["sampling_rate"]:.4f} Hz', 
    "Difference between .wav file and lsl timestamps durations": f'{mic_vars["lsl_wav_duration_diff"]:.4f} sec', 
    "Number of NaN's": f'{mic_vars["num_NaN"]}',
    "Percent of NaN's": f'{mic_vars["percent_NaN"]:.4f}%',
    "Mic samples first quartile": f'{mic_vars["quan25"]:.4f}',
    "Mic samples third quartile": f'{mic_vars["quan75"]:.4f}',
    "Mic samples std": f'{mic_vars["std"]:.4f}',
    "Mic samples min": f'{mic_vars["min"]:.4f}',
    "Mic samples max": f'{mic_vars["max"]:.4f}'
}

## 6. Webcam

In [ ]:
webcam_vars, cam_df = webcam_qc(xdf_filename=xdf_filename,
                                stim_df=stim_df,
                                video_file=video_filename, task='RestingState')

In [ ]:
webcam_print = {
    "Effective sampling rate": f'{webcam_vars["sampling_rate"]:.4f} Hz', 
    "Collected full resting state": webcam_vars["collected_full_RestingState"], 
    "Percent of frames with face detected": f'{webcam_vars["face_perc"]:.4%}'
}

In [ ]:
webcam_print

## 7. ET

In [ ]:
et_vars, et_df = et_qc(xdf_filename = xdf_filename, stim_df = stim_df, task='RestingState')

In [ ]:
et_print = {
    "Effective sampling rate": f'{et_vars["sampling_rate"]:.4f} Hz', 
    "Flag: all coordinates have the same % validity within each measure (LR, gaze point/origin/diameter)": et_vars["flag1"], 
    "Flag: % of NaNs is the same between coordinate systems (UCS and TBCS (gaze origin) and between UCS and display area (gaze point))": et_vars["flag2"],
    "Mean difference in percent valid data between right and left eyes": f'{et_vars["LR_mean_diff"]:.4f}%',
    "Percent of data with gaze point differences of over 0.2 mm": f'{et_vars["percent_over02"]:.4f}%'
}

## Stream Durations 

### functions are in utils.py (but should be called in the report)

In [ ]:
df_map = {
    'et': et_df,
    'ps': ps_df,
    'mic': mic_df,
    'cam': cam_df
    }
    # 'eeg': eeg_df

In [ ]:
duration_vars = {"Durations of each modality + comparison to expected duration:": 
    get_durations(xdf_path=xdf_filename, task='Experiment', df_map = df_map, stim_df = stim_df)}

In [ ]:
duration_print = duration_vars

In [ ]:
# i wont run these but they are here for reference
# get_durations('RestingState', xdf_filename)
# get_durations('CampFriend', xdf_filename)
# get_durations('SocialTask', xdf_filename)
# whole_durations(xdf_filename)

## LSL Problem

In [ ]:
lsl_vars = lsl_problem_qc(xdf_filename, df_map=df_map, stim_df=stim_df, modality_to_plot='et')

In [ ]:
lsl_print = {
    "Percent of missing data in entire experiment": lsl_vars["percent_loss"],
    "Percent of missing data before social task offset": lsl_vars["loss_before_social_task"]
}

# Copy

In [ ]:
copy = {
    'EEG': 'Data preprocessed by performing <b>line noise removal</b>, <b>robust referencing</b>, and <b>bad channel detection/interpolation</b> using PyPrep pipeline. First, the pipeline applies a notch filter at 60 Hz and its harmonics to remove power line noise. Then, it performs <b>robust average referencing</b>, where it detects bad channels, interpolates them using surrounding signals, and computes a median-based reference across EEG channels. This ensures a stable reference even in the presence of noisy electrodes. Independent components analysis (ICA) was then performed to identify and remove ocular and muscle artifacts. The final output is a cleaned EEG dataset with a consistent reference, ready for further analysis.',
    'ET': 'In the eye tracking data, gaze origin, gaze point, and pupil diameter are reported, for the left and right eyes. Gaze origin describes the point in 3D space where the gaze vector starts, and is reported in both the user coordinate system (UCS) and the trackbox coordinate system (TBCS). Gaze point describes the spot on the screen where the participant is looking, for each eye. Gaze origin is reported in 3D space in the UCS and the display area. The percent of data for which the distance between the gaze point of the left and right eye is more than 0.2 mm is reported below.',
    'ECG': "The data was processed using the neurokit2 automated pipeline for preprocessing ECG signals. ECG data is cleaned using the default parameters that apply fifth order 0.5 Hz high-pass butterworth filter followed by 50 Hz powerline filtering. Neurokit2 was also used to calculate signal quality indices (SQI) based on Zhao et.al (2018). SQIs calculated were kurtosis (kSQI), power spectrum distribution of QRS wave (pSQI) and relative power in baseline (basSQI). kSQI was calculated using the Fisher's method. pSQI and basSQI were calculated using the Welch method.",
    'EDA': "Electrodermal Activity (EDA) is measured in microSiemens from a standard EDA sensor. EDA data quality metrics calculated from raw and clean data using neurokit2 python package. The data is processed using the neurokit2 automated pipeline for preprocessing EDA signal. EDA data is cleaned using the default parameters that apply fourth order butterworth filter. Additionally, the automated preprocessing pipeline was used to decompose the EDA signal into the tonic component, Skin Conductance Level (SCL) and the phasic component, Skin Conductance Response (SCR). The decomposition was done using default parameters that perform high-pass filtering with a cutoff frequency of 0.05Hz on EDA signal.",
    'RSP': "Respiration data is measured in volts from a piezoelectric sensor. Respiration data quality metrics are calculated both from raw and cleaned data, where the neurokit2 package was used to clean the data using the default method of a second order 0.05-3 Hz bandpass Butterworth filter, as in Khodadad et al, 2018. Neurokit2 was also used to extract the peaks and troughs of detected breaths. Respiration rate was calculated using the cross correlation method, which was more consistent to the raw data than the other trough method. Baseline drift was quantified as the standard deviation of the data after applying a lowpass filter to raw data.",
    'MIC': "For audio data, it is important to confirm that the distribution of microphone samples (see graph below) is relatively centered around 0, with no outlier peaks and no values above 32,000 or below -32,000.",
    'WEBCAM': "The quality of the webcam data is assessed by checking the percentage of frames where a face is detected. If a face is not detected in more than 50 percent of the frames, it may indicate issues with the recording or participant positioning.",
    'Stream Durations': "",
    'LSL': "While the difference between consecutive LSL time stamps are expected to consistently be equal to the inverse of the sampling rate, it was found that there were longer gaps between some LSL time stamps for some data modalities, indicating a potential system or other technical error during data collection. Below, the percentage of missing data for each data modality due to these LSL time stamp gaps is shown. The amount of data in seconds for each modality is also shown below, and each duration is then compared to the expected duration of the experiment."
}

# Second Report with formatting

In [ ]:
# Modalities and corresponding data
metric_names = ["EEG", "ECG", "EDA", "RSP", "MIC","ET", "WEBCAM", "Stream Durations", "LSL"]
metrics_list = [eeg_print, ecg_print, eda_print, rsp_print, mic_print, et_print, webcam_print, duration_print, lsl_print]
# metric_names = ["ECG", "EDA","WEBCAM"]
# metrics_list = [ecg_print, eda_print,  webcam_print]

# PDF structure
parent_folder = xdf_filename.split('mobi')[0]
pdf_path = f"{parent_folder}QCReport.pdf"
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

# Define subtitle style if not already done
subtitle_style = ParagraphStyle(
    name="Subtitle",
    parent=styles["Heading2"],
    fontSize=14,
    leading=16,
    textColor="gray",
    spaceAfter=12,
    alignment=1  # Centered
)

# page number function
def add_page_number(canvas, doc):
    page_num = f'{canvas.getPageNumber()}'
    canvas.setFont("Helvetica", 9)
    canvas.drawRightString(570, 20, page_num)  # (x, y) from bottom-left


elements.append(Paragraph(f"Subject Report: {subject}", styles["Title"]))
elements.append(Paragraph(f"Collection Date: {get_collection_date(xdf_filename)}", subtitle_style))
elements.append(Spacer(1, 12))

# Format each metrics dict
for name, metrics in zip(metric_names, metrics_list):
    elements.append(Paragraph(name, styles["Heading2"]))
    elements.append(Paragraph(copy[name], styles["Normal"]))
    elements.append(Spacer(1, 12))

    for k, v in metrics.items():
        if isinstance(v, pd.DataFrame):

            data = [v.columns.tolist()] + v.values.tolist()  # Include headers
            table = Table(data, repeatRows=1)
            table.hAlign = 'LEFT'

            table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
                ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ]))
            elements.append(Paragraph(k, styles['Normal']))
            elements.append(Spacer(1, 12))

            elements.append(table)
            elements.append(Spacer(1, 12))
        else:
            text = f"<b>{k}:</b> {v}" if isinstance(v, float) else f"<b>{k}:</b> {v}"
            elements.append(Paragraph(text, styles["Normal"]))
    elements.append(Spacer(1, 12))

    # images
    folder = "report_images"
    image_keyword = name.lower()
    if os.path.exists(folder):
        for fname in sorted(os.listdir(folder)):
            if image_keyword in fname.lower() and subject in fname:
                image_path = os.path.join(folder, fname)

                # get dimensions
                image_reader = ImageReader(image_path)
                orig_width, orig_height = image_reader.getSize()
                print(orig_width, orig_height)
                if orig_width > 4000:
                    img = Image(image_path, width=orig_width/9, height=orig_height/9)
                elif orig_width > 3000:
                    img = Image(image_path, width=orig_width/7, height=orig_height/7)
                elif orig_width > 2000:
                    img = Image(image_path, width = orig_width/5, height = orig_height/5)
                elif orig_width > 1400:
                    img = Image(image_path, width=orig_width/3, height=orig_height/3)
                else:
                    img = Image(image_path, width=orig_width/2, height=orig_height/2)  # Adjust size as needed
                elements.append(img)
                elements.append(Spacer(1, 12))



doc.build(elements, onFirstPage = add_page_number, onLaterPages = add_page_number)
print(f'PDF created: {pdf_path}')

In [ ]:
# Modalities and corresponding data
metric_names = ["EEG", "ECG", "EDA", "RSP", "MIC","ET", "WEBCAM", "Stream Durations", "LSL"]
metrics_list = [eeg_print, ecg_print, eda_print, rsp_print, mic_print, et_print, webcam_print, duration_print, lsl_print]


# PDF structure
parent_folder = xdf_filename.split('mobi')[0]
pdf_path = f"{parent_folder}QCReport.pdf"
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

# Define subtitle style if not already done
subtitle_style = ParagraphStyle(
    name="Subtitle",
    parent=styles["Heading2"],
    fontSize=14,
    leading=16,
    textColor="gray",
    spaceAfter=12,
    alignment=1  # Centered
)

# page number function
def add_page_number(canvas, doc):
    page_num = f'{canvas.getPageNumber()}'
    canvas.setFont("Helvetica", 9)
    canvas.drawRightString(570, 20, page_num)  # (x, y) from bottom-left


elements.append(Paragraph(f"Subject Report: {subject}", styles["Title"]))
elements.append(Paragraph(f"Collection Date: {get_collection_date(xdf_filename)}", subtitle_style))
elements.append(Spacer(1, 12))

# Format each metrics dict
for name, metrics in zip(metric_names, metrics_list):
    elements.append(Paragraph(name, styles["Heading2"]))
    elements.append(Paragraph(copy[name], styles["Normal"]))
    elements.append(Spacer(1, 12))

    for k, v in metrics.items():
        if isinstance(v, pd.DataFrame):
            data = [v.columns.tolist()] + v.values.tolist()  # Include headers
            table = Table(data, repeatRows=1)
            table.hAlign = 'LEFT'

            table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
                ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ]))
            elements.append(Paragraph(k, styles['Normal']))
            elements.append(Spacer(1, 12))

            elements.append(table)
            elements.append(Spacer(1, 12))
        else:
            text = f"<b>{k}:</b> {v}" if isinstance(v, float) else f"<b>{k}:</b> {v}"
            elements.append(Paragraph(text, styles["Normal"]))
    elements.append(Spacer(1, 12))

    # images
    folder = "report_images"
    image_keyword = name.lower()
    if os.path.exists(folder):
        row_images = []
        for fname in sorted(os.listdir(folder)):
            if image_keyword in fname.lower() and subject in fname:
                image_path = os.path.join(folder, fname)

                # get dimensions
                image_reader = ImageReader(image_path)
                orig_width, orig_height = image_reader.getSize()
                if orig_width > 4000:
                    img = Image(image_path, width=orig_width/9, height=orig_height/9)
                elif orig_width > 3000:
                    img = Image(image_path, width=orig_width/7, height=orig_height/7)
                elif orig_width > 2000:
                    img = Image(image_path, width=orig_width/5, height=orig_height/5)
                elif orig_width > 1400:
                    img = Image(image_path, width=orig_width/3, height=orig_height/3)
                else:
                    img = Image(image_path, width=orig_width/2, height=orig_height/2)

                # If 'rsp' in image name, add to a row
                if 'rsp' in fname.lower():
                    img.drawWidth = 250
                    img.drawHeight = 200
                    row_images.append(img)
                    
                    if len(row_images) == 2:
                        table = Table([row_images], hAlign = 'CENTER', colWidths=[280,280])
                        table.setStyle(TableStyle([('ALIGN', (0, 0), (0, -1), 'LEFT')]))
                        elements.append(table)
                        elements.append(Spacer(1, 12))
                        row_images = []

                else:
                    elements.append(img)
                    elements.append(Spacer(1, 12))

        # If there's an unmatched image left in the row buffer
        if row_images:
            elements.append(Table([row_images], hAlign='CENTER', colWidths='*'))
            elements.append(Spacer(1, 12))
            

doc.build(elements, onFirstPage = add_page_number, onLaterPages = add_page_number)
print(f'PDF created: {pdf_path}')